In [10]:
import numpy as np
from scipy.spatial import ConvexHull
import MDAnalysis as mda


u = mda.Universe('trajectory/1jwp_noh.pdb')
selected_atoms = u.select_atoms('name CA and (resid 236 to 237 or resid 215 to 224 or resid 242 to 245 or resid 268 to 284)')
# Get the positions of the selected atoms
points = selected_atoms.positions

# Calculate the convex hull
hull = ConvexHull(points)

In [11]:
# Extract vertices and faces
vertices = points[hull.vertices]
faces = hull.simplices

In [7]:
def export_for_pymol(points, faces, filename='convex_hull_open.pml', transparency=0.5, line_width=2.0):
    with open(filename, 'w') as file:
        # Write the header for the PyMOL script
        file.write("from pymol.cgo import *\nfrom pymol import cmd\n\n")
        # Set color to black for the edges
        black = ["COLOR", 0.0, 0.0, 0.0]  # RGB color for black
        # Initialize the obj list with BEGIN statement for TRIANGLES
        obj = []

        # Iterate over each face to add vertices to the obj list
        for face in faces:
            obj.append("BEGIN")
            obj.append("TRIANGLES")
            obj.extend(["ALPHA", transparency])
            for vertex in face:
                x, y, z = points[vertex]
                obj.extend(["VERTEX", x, y, z])
            obj.append("END") 
            obj.append("BEGIN")
            obj.append("LINES")
            for i in range(len(face)):
                # Get the current vertex and the next vertex to form an edge
                start_vertex = points[face[i]]
                end_vertex = points[face[(i + 1) % len(face)]]  # Wrap around to the first vertex
                obj.extend(["VERTEX", start_vertex[0], start_vertex[1], start_vertex[2]])
                obj.extend(["VERTEX", end_vertex[0], end_vertex[1], end_vertex[2]])
            obj.append("END")

        # Convert obj list to a string representation for PyMOL script
        obj_str = ', '.join([str(item) if isinstance(item, int) else f"{item}" for item in obj])
        
        # Write the obj definition to the file
        file.write(f"obj = [{obj_str}]\n")
        
        # Include the command to load the CGO object into PyMOL
        file.write("cmd.load_cgo(obj, 'convex_hull')\n")

# Use the function to generate a PyMOL script, using original points and faces
export_for_pymol(points, faces)


In [12]:
def export_for_chimerax(points, hull, filename='convex_hull_open.obj'):
    with open(filename, 'w') as file:
        # Write vertices to the file
        for vertex in hull.points:
            file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
        
        # Write faces to the file
        for simplex in hull.simplices:
            # OBJ file indices are 1-based, not 0-based and here we reference the hull vertices
            file.write(f"f {simplex[0] + 1} {simplex[1] + 1} {simplex[2] + 1}\n")

# Use the function to generate an OBJ file, using hull points and simplices
export_for_chimerax(hull.points, hull)